## Lab 1.1 - Analyzing Real Estate Data in St. Petersburg
We have data from Yandex.Realty classified https://realty.yandex.ru containing real estate listings for apartments in St. Petersburg and Leningrad Oblast from 2016 till the middle of August 2018. In this Lab you'll learn how to work with datasets in python using pandas library for data analysis using the provided data. 
For datasets like this it becomes hard to use Excel because of its size. If you learn how to work with pandas you'll be able to do basic analytical research much faster than in Excel.
To execute code IPython notebook click on the Cell and press Shift-Enter.
If you don't have any experience with Jupyter notebooks, please look at this tutorial: https://mybinder.org/v2/gh/ipython/ipython-in-depth/master?filepath=binder/Index.ipynb

### Main objectives
After successful completion of the lab work students will be able to:
-	Analyze large datasets that don't work well in Excel using IPython notebook and pandas library
-   Find outliers and remove them in the datasets
-	Calculate different statistics on data fields


### Tasks
-	Calculate median and mean sell and rent prices in St. Petersburg
-   Find outliers: too cheap or too expensive apartments which seem to be a wrong data. Remove them from the dataset.
-	Find houses with the most cheapest and most expensive prices per square meter.
-   Analyze how many rent offers have the commission and what is the most popular commission



In [2]:
# let's import pandas library and set options to be able to view data right in the browser
# Assign pd for a short alias for pandas library
import pandas as pd
# Call set_option function fron pandas library to edit them limit for number of the displayed
# columns, width of all columns and maximum width of one column
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [3]:
# let's look what is inside our data file to understand how we can export it. 
# We can use linux command "head" which shows the first row in the file. 
# In IPython notebooks you can execute linux commands using "!" prefixe before the command name. 
# This mechanism is called "IPython magic"
!head -n 3 spb.real.estate.archive.2018.tsv

offer_id	first_day_exposition	last_day_exposition	last_price	floor	open_plan	rooms	studio	area	kitchen_area	living_area	agent_fee	renovation	offer_type	category_type	unified_address	building_id
5490504238877583424	2014-11-27T00:00:00+03:00	2016-03-14T00:00:00+03:00	3676740.0	19	False	1	False	27.4	16.56	16.56		0.0	1	2	Россия, Санкт-Петербург, улица Шкапина, 9-11	3552413344225333068
5490504238903444021	2014-11-27T00:00:00+03:00	2016-04-28T00:00:00+03:00	7038880.0	2	False	3	False	85.84	19.64	41.2		0.0	1	2	Россия, Санкт-Петербург, Кушелевская дорога, 3к4	7274736308217971906


In [4]:
# if you don't remember what linux command does (i.e. what is -n), you can always look for help
!head --help

Usage: head [OPTION]... [FILE]...
Print the first 10 lines of each FILE to standard output.
With more than one FILE, precede each with a header giving the file name.

With no FILE, or when FILE is -, read standard input.

Mandatory arguments to long options are mandatory for short options too.
  -c, --bytes=[-]NUM       print the first NUM bytes of each file;
                             with the leading '-', print all but the last
                             NUM bytes of each file
  -n, --lines=[-]NUM       print the first NUM lines instead of the first 10;
                             with the leading '-', print all but the last
                             NUM lines of each file
  -q, --quiet, --silent    never print headers giving file names
  -v, --verbose            always print headers giving file names
  -z, --zero-terminated    line delimiter is NUL, not newline
      --help     display this help and exit
      --version  output version information and exit

NUM may have a mu

In [5]:
!ls

 cleaned_dataset.csv		      'New lab_1.ipynb'
'elab2_building_model (3).ipynb'       scaler_x.pkl
 lab1_1_EDA_real_estate_data.ipynb     scaler_y.pkl
'lab1_2_visualization (1) (1).ipynb'   spb.real.estate.archive.2018.tsv
 lab1_2_visualization.ipynb	       spb.real.estate.archive.sample5000.tsv
'lab2_building_model (1).ipynb'        Untitled19.ipynb
 meee2.ipynb			       Untitled.ipynb
 model.pkl


In [6]:
# we see that it's tab separated file, we can load it with pd.read_table method
# load our dataset and see which data it contains.
# HINT: you can type 'Tab' after starting typing the filename and IPython will automatically fill it for you
# After this command a new variable will be created (spb_df) containing pandas dataframe with data in our file
# You can think of dataframe structure as an analogue of Excel sheet
spb_df = pd.read_table('spb.real.estate.archive.2018.tsv')

In [7]:
# HINT: if you don't know which arguments any function takes or what it does you can check the documenation right 
# in the notebook just by adding "?" after the function name. Let's look at pandas read_table function:
pd.read_table?

Signature:
pd.read_table(
    filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]',
    *,
    sep: 'str | None | lib.NoDefault' = <no_default>,
    delimiter: 'str | None | lib.NoDefault' = None,
    header: "int | Sequence[int] | None | Literal['infer']" = 'infer',
    names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>,
    index_col: 'IndexLabel | Literal[False] | None' = None,
    usecols=None,
    squeeze: 'bool | None' = None,
    prefix: 'str | lib.NoDefault' = <no_default>,
    mangle_dupe_cols: 'bool' = True,
    dtype: 'DtypeArg | None' = None,
    engine: 'CSVEngine | None' = None,
    converters=None,
    true_values=None,
    false_values=None,
    skipinitialspace: 'bool' = False,
    skiprows=None,
    skipfooter: 'int' = 0,
    nrows: 'int | None' = None,
    na_values=None,
    keep_default_na: 'bool' = True,
    na_filter: 'bool' = True,
    verbose: 'bool' = False,
    skip_blank_lines: 'bool' = True,
    parse_dates=False,
    in

In [8]:
# let's check how much data in terms of rows we have where each row represent 
# one offer from Yandex.Realty
len(spb_df)

429187

In [9]:
# let's look at random sample of size 10 of the loaded dataset to understand what's inside
spb_df.sample(10)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
213722,8932145010549726208,2017-07-23T00:00:00+03:00,2018-05-24T00:00:00+03:00,13000.0,5,False,1,False,35.00,8.0,18.0,100.0,NaN,2,2,"Россия, Ленинградская область, Тосненское городское поселение, Тосно, проспект Ленина, 57",2739938529422030765
119158,1943688282461195777,2016-12-27T00:00:00+03:00,2017-01-03T00:00:00+03:00,12000.0,22,False,0,True,26.00,NaN,17.0,100.0,3.0,2,2,"Россия, Ленинградская область, Всеволожский район, посёлок Мурино, бульвар Менделеева, 3",5479032906071076088
226127,1372961398720901120,2017-10-03T00:00:00+03:00,2018-06-07T00:00:00+03:00,4200000.0,7,False,1,False,43.00,7.0,17.5,NaN,3.0,1,2,"Россия, Санкт-Петербург, проспект Просвещения, 68к1",561327474932235776
40021,5389167,2016-02-13T00:00:00+03:00,2016-02-20T00:00:00+03:00,16000.0,10,False,1,False,38.00,10.2,17.0,50.0,0.0,2,2,"Россия, Санкт-Петербург, посёлок Парголово, Приозерское шоссе, 16к4",1318980229802740776
406576,7568841808380499533,2018-07-24T00:00:00+03:00,2018-08-02T00:00:00+03:00,8900000.0,12,False,2,False,70.00,12.0,39.0,NaN,NaN,1,2,"Россия, Санкт-Петербург, улица Веденеева, 2",146662967299063309
199459,4493803187535074304,2017-01-14T00:00:00+03:00,2018-02-08T00:00:00+03:00,1650000.0,4,False,1,False,42.00,9.0,18.0,NaN,8.0,1,2,"Россия, Ленинградская область, Приозерск, улица Гоголя, 7",3248623912739769382
222119,7621245106430172499,2017-09-26T00:00:00+03:00,2017-10-02T00:00:00+03:00,5950000.0,11,False,2,False,59.00,14.0,30.0,1.0,0.0,1,2,"Россия, Ленинградская область, Всеволожский район, Заневское городское поселение, деревня Кудрово, Австрийская улица, 4к1",8305710144320377257
11270,5867283855211936768,2015-09-03T00:00:00+03:00,2016-11-09T00:00:00+03:00,3955875.0,3,False,2,False,75.35,16.4,29.3,NaN,0.0,1,2,"Россия, Ленинградская область, Тосненский район, Фёдоровское сельское поселение, деревня Фёдоровское, Почтовая улица, 9к1",3496334200837814708
232196,1440420981569272832,2015-11-19T00:00:00+03:00,2018-03-13T00:00:00+03:00,5760000.0,6,False,3,False,57.00,10.0,35.0,NaN,3.0,1,2,"Россия, Санкт-Петербург, Конная улица, 10",3242117185209680048
377707,5565310413830655172,2018-06-19T00:00:00+03:00,2018-06-26T00:00:00+03:00,80000.0,1,False,0,True,113.00,NaN,NaN,100.0,NaN,2,2,"Россия, Санкт-Петербург, набережная реки Фонтанки, 83Д",5091220263802463564


In [10]:
len(spb_df)

429187

In [11]:
spb_df.sample(5000).to_csv('spb.real.estate.archive.sample5000.tsv', index = False, sep='\t')

In [12]:
!pwd

/home/jovyan/2ML_Course


### Analyze prices for rent and sell
We see that we have both RENT and SELL data. 
<p>offer_type column contains data to distinct them, 2 stands for RENT, 1 for SELL


#### Analyze rent prices

In [13]:
# at first let's split our dataframe to rent and sell
# Select some part of dataframe according to conditions in brackets
rent_df = spb_df[spb_df.offer_type == 2]
sell_df = spb_df[spb_df.offer_type == 1]
# .format function allows you to put variales' values instead of {} in the final string
print("Total rent data size: {}".format(len(rent_df)))
print("Total sell data size: {}".format(len(sell_df)))

Total rent data size: 171186
Total sell data size: 258001


In [14]:
# let's check median and price for renting apartments
# .last_price indicates that you use data just from the column named last_price
rent_median_price = rent_df.last_price.median()
rent_mean_price = rent_df.last_price.mean()
print("Rent median price: {}".format(rent_median_price))
print("Rent mean price: {}".format(rent_mean_price))

Rent median price: 25000.0
Rent mean price: 31719.256638977487


In [15]:
# Just to look at data of one column you can select this column. 
# 'head' function will take only first rows 
rent_df.last_price.head(10)

1086    20000.0
1096    40000.0
1097    45000.0
1149    21000.0
1514    25000.0
1515    35000.0
1516    22000.0
1537    20000.0
1555    26000.0
1695    25000.0
Name: last_price, dtype: float64

#### Let's look at the most expensive apartments in rent

In [16]:
# .max function returns the maximum value across all rows
max_price_rent = rent_df.last_price.max()

In [17]:
print("Max price in rent: {}".format(max_price_rent))

Max price in rent: 4900000.0


In [18]:
# Now select all offers where price is equal to maximum value across all rent prices
rent_df_max = rent_df[rent_df.last_price == max_price_rent]

In [19]:
rent_df_max

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
123675,5594900220237251840,2017-01-16T00:00:00+03:00,2017-01-16T00:00:00+03:00,4900000.0,3,False,3,False,55.4,5.5,42.1,NaN,0.0,2,2,"Россия, Санкт-Петербург, Новочеркасский проспект, 36",7844665691371923607


We can see that it was probably a mistake. It's hard to imagine why an aparment could cost 4.9 mln rubles per month. To analyze it let's look at the random sample of rent offers with prices of more than 1 mln rubles.

In [20]:
# Knowing real estate market in St.Petersburg we understand 
# that offers with rent prices over 1 million rubles are outliers
outliers_rent = rent_df[rent_df.last_price > 1000000]
print("Number of outliers in rent: {}".format(len(outliers_rent)))

Number of outliers in rent: 22


In [21]:
outliers_rent.sample(5)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
123451,4266909851082118912,2017-01-14T00:00:00+03:00,2017-05-27T00:00:00+03:00,3100000.0,2,False,1,False,39.0,7.0,20.0,NaN,3.0,2,2,"Россия, Санкт-Петербург, Петергоф, Озерковая улица, 39к1",6053955221850724766
12990,3988133,2015-09-19T00:00:00+03:00,2016-04-13T00:00:00+03:00,3450000.0,19,False,1,True,30.0,NaN,NaN,NaN,3.0,2,2,"Россия, Санкт-Петербург, посёлок Парголово, Заречная улица, 19к1",2079266574874175179
34888,5199154,2016-01-25T00:00:00+03:00,2016-01-26T00:00:00+03:00,3800000.0,3,False,2,False,51.0,6.0,30.5,NaN,0.0,2,2,"Россия, Санкт-Петербург, Пискарёвский проспект, 149",7757194067653498771
10707,3751871,2015-08-27T00:00:00+03:00,2016-05-12T00:00:00+03:00,2100000.0,2,False,2,False,49.0,7.0,NaN,90.0,3.0,2,2,"Россия, Санкт-Петербург, проспект Народного Ополчения, 179к1",2789010111420303027
123675,5594900220237251840,2017-01-16T00:00:00+03:00,2017-01-16T00:00:00+03:00,4900000.0,3,False,3,False,55.4,5.5,42.1,NaN,0.0,2,2,"Россия, Санкт-Петербург, Новочеркасский проспект, 36",7844665691371923607


In [22]:
# We see that all of these offers are results of user mistakes.
# Pause to think what kind of mistakes might cause such numbers?
# Let's now look at rent offers below 1M but more than 500K

In [23]:
rent_df[(rent_df.last_price > 500000) & (rent_df.last_price < 1000000)]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
83621,6302977522903470578,2015-05-28T00:00:00+03:00,2017-04-26T00:00:00+03:00,550000.0,6,False,4,False,180.0,18.0,122.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507
89836,6302977522901450489,2016-09-14T00:00:00+03:00,2016-09-15T00:00:00+03:00,800000.0,2,False,2,False,80.0,20.0,40.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Большая Морская улица, 27",8010764606439654926
106222,4377907628678297857,2016-11-07T00:00:00+03:00,2017-03-13T00:00:00+03:00,600000.0,6,False,4,False,187.0,NaN,127.0,100.0,1.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507
172166,3376106655979657514,2017-02-24T00:00:00+03:00,2017-05-24T00:00:00+03:00,600000.0,6,False,4,False,200.0,NaN,103.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к2",7861463534572863508
172950,1896730477252952293,2016-10-31T00:00:00+03:00,2017-05-26T00:00:00+03:00,650000.0,6,False,4,False,190.0,20.0,102.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507
253075,9174606537144832259,2017-11-16T00:00:00+03:00,2018-08-20T00:00:00+03:00,600000.0,1,False,4,False,165.0,NaN,115.0,NaN,6.0,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355
321216,2596865999798823681,2018-03-31T00:00:00+03:00,2018-04-04T00:00:00+03:00,575000.0,3,False,4,False,120.0,NaN,NaN,50.0,1.0,2,2,"Россия, Санкт-Петербург, Кременчугская улица, 17к3",9078299753075959719
323711,9106659135022232833,2018-04-03T00:00:00+03:00,2018-06-15T00:00:00+03:00,600000.0,1,False,4,False,90.0,13.0,54.0,0.0,11.0,2,2,"Россия, Санкт-Петербург, Большой Сампсониевский проспект, 33/1",192308642065385452
336616,8867625833179319559,2018-04-21T00:00:00+03:00,2018-08-09T00:00:00+03:00,600000.0,1,False,4,False,117.0,20.0,83.0,0.0,NaN,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355
419166,5152264856292210972,2018-02-01T00:00:00+03:00,2018-08-16T00:00:00+03:00,600000.0,1,False,4,False,140.0,95.0,100.0,100.0,NaN,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355


In [24]:
# for these cases sometimes it seems that the price is correct - in the unified_address column we
# see that these offers are from elite houses
# like here: https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20проспект%20Динамо%2C%206/snyat/kvartira/
# but for some of them price seems to be 10 times more than it should be, like here
# https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20Большая%20Морская%20улица%2C%2027/snyat/kvartira/
# let's find these cases

### Calculate price per square meter, get median prices for house and find outliers with the help of this

In [25]:
# Create a new column price_per_sq_m and calculate price per sq m
rent_df['price_per_sq_m'] = rent_df.last_price/rent_df.area

/tmp/ipykernel_456/816257644.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rent_df['price_per_sq_m'] = rent_df.last_price/rent_df.area


In [26]:
rent_df.head(5)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m
1086,1100202,2015-01-24T00:00:00+03:00,2016-01-19T00:00:00+03:00,20000.0,7,False,1,False,28.0,8.0,20.0,100.0,3.0,2,2,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",6719200042030752141,714.285714
1096,5028522699704970725,2015-01-25T00:00:00+03:00,2016-01-20T00:00:00+03:00,40000.0,4,False,2,False,72.0,10.0,44.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, набережная реки Фонтанки, 66",3406104650917898355,555.555556
1097,5028522699709767236,2015-01-25T00:00:00+03:00,2016-01-21T00:00:00+03:00,45000.0,2,False,2,False,55.0,NaN,33.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Большой проспект П.С., 102",2357095685903407888,818.181818
1149,1100874,2015-01-28T00:00:00+03:00,2016-01-16T00:00:00+03:00,21000.0,5,False,1,False,35.0,8.0,18.0,NaN,11.0,2,2,"Россия, Санкт-Петербург, Лужская улица, 8",5887402119500981764,600.000000
1514,875413,2015-02-13T00:00:00+03:00,2016-01-23T00:00:00+03:00,25000.0,10,False,1,True,44.0,15.0,18.0,90.0,10.0,2,2,"Россия, Санкт-Петербург, Учительская улица, 3",6527910144166454997,568.181818


In [27]:
# find what's median and mean price per square meter in rent
median_price_per_sq_m = rent_df.price_per_sq_m.median()
mean_price_per_sq_m = rent_df.price_per_sq_m.mean()
print("Median price per sq m in rent: {}".format(median_price_per_sq_m))
print("Mean price per sq m in rent: {}".format(mean_price_per_sq_m))

Median price per sq m in rent: 550.0
Mean price per sq m in rent: 600.1110692967684


#### Find median price per sq m per house

In [28]:
# .groupby function groups dataframe by a selected column (unified_address)
# For each address we then take a median of price per sq m and return a new dataframe 
# with a result
house_rent_df = rent_df.groupby('unified_address').price_per_sq_m.median().reset_index()

In [29]:
house_rent_df.head(5)

,unified_address,price_per_sq_m
0,"Россия, Ленинградская область, Бокситогорск, Комсомольская улица, 5А",171.428571
1,"Россия, Ленинградская область, Бокситогорск, Южная улица, 15",250.000000
2,"Россия, Ленинградская область, Бокситогорск, улица Металлургов, 7",164.948454
3,"Россия, Ленинградская область, Бокситогорск, улица Павлова, 37",138.888889
4,"Россия, Ленинградская область, Бокситогорский район, Пикалёво, 6-й микрорайон, 25",195.652174


In [30]:
# Rename the column with a 'rename' function in this new dataframe
house_rent_df.rename(columns = {'price_per_sq_m': 'house_price_sqm_median'}, inplace = True)

#### Merge rent data with house median prices and inspect outliers

In [31]:
# With a 'merge' function join two dataframes.
# As these 2 dataframes have only one common column name (unified_address) it will merge dataframes by it
rent_df = rent_df.merge(house_rent_df)

In [32]:
# Assign outliers if price per sq m is 5 times higher than the median value in the house
outliers = rent_df[(rent_df.price_per_sq_m/rent_df.house_price_sqm_median) > 5]

In [33]:
print(len(outliers))

49


In [34]:
outliers.sample(10)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
84160,5148571494042384844,2016-03-15T00:00:00+03:00,2016-06-06T00:00:00+03:00,240000.0,4,False,2,False,47.0,9.0,30.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, улица Брянцева, 28",130991341981241322,5106.382979,478.260870
50140,6302977522901450489,2016-09-14T00:00:00+03:00,2016-09-15T00:00:00+03:00,800000.0,2,False,2,False,80.0,20.0,40.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Большая Морская улица, 27",8010764606439654926,10000.000000,841.121495
2471,5565310413830597579,2018-06-19T00:00:00+03:00,2018-06-26T00:00:00+03:00,180000.0,2,False,0,True,27.0,NaN,NaN,100.0,NaN,2,2,"Россия, Санкт-Петербург, Кондратьевский проспект, 64к9",1081182048329084375,6666.666667,638.888889
117316,8256452766188685912,2018-05-11T00:00:00+03:00,2018-08-03T00:00:00+03:00,400000.0,3,False,2,False,85.0,30.0,40.0,50.0,1.0,2,2,"Россия, Санкт-Петербург, Итальянская улица, 11",15264832366839851,4705.882353,933.333333
74207,6589696843731071745,2018-03-12T00:00:00+03:00,2018-04-02T00:00:00+03:00,2500000.0,2,False,4,False,150.0,53.0,47.0,50.0,NaN,2,2,"Россия, Санкт-Петербург, Волховский переулок, 4",4642914072625498389,16666.666667,887.850467
80578,1523340115924662017,2018-07-26T00:00:00+03:00,2018-08-20T00:00:00+03:00,1752250.0,21,False,2,False,38.0,13.0,25.0,50.0,10.0,2,2,"Россия, Санкт-Петербург, посёлок Парголово, улица Фёдора Абрамова, 23к1",4731794338099306507,46111.842105,461.538462
24222,3988133,2015-09-19T00:00:00+03:00,2016-04-13T00:00:00+03:00,3450000.0,19,False,1,True,30.0,NaN,NaN,NaN,3.0,2,2,"Россия, Санкт-Петербург, посёлок Парголово, Заречная улица, 19к1",2079266574874175179,115000.000000,571.428571
117317,8256452766188712792,2018-05-11T00:00:00+03:00,2018-08-20T00:00:00+03:00,400000.0,3,False,3,False,85.0,15.0,55.0,50.0,1.0,2,2,"Россия, Санкт-Петербург, Итальянская улица, 11",15264832366839851,4705.882353,933.333333
133370,5594900220237251840,2017-01-16T00:00:00+03:00,2017-01-16T00:00:00+03:00,4900000.0,3,False,3,False,55.4,5.5,42.1,NaN,0.0,2,2,"Россия, Санкт-Петербург, Новочеркасский проспект, 36",7844665691371923607,88447.653430,695.378151
146984,2484217340613565185,2018-05-26T00:00:00+03:00,2018-06-02T00:00:00+03:00,300000.0,15,False,2,False,60.0,12.0,35.0,50.0,10.0,2,2,"Россия, Санкт-Петербург, улица Шелгунова, 7к1",4224014818886931677,5000.000000,500.000000


#### Clean data from the outliers

In [35]:
# Let's create a new dataframe with clean data.
# With ~ we can select the data which does NOT correspond to the required conditions
rent_df_cleaned = rent_df[~((rent_df.price_per_sq_m/rent_df.house_price_sqm_median) > 5)]

In [36]:
# check that in the middle segment everything is fine now
rent_df_cleaned[(rent_df_cleaned.last_price > 500000) & (rent_df_cleaned.last_price < 1000000)]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
95391,3376106655979657514,2017-02-24T00:00:00+03:00,2017-05-24T00:00:00+03:00,600000.0,6,False,4,False,200.0,NaN,103.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к2",7861463534572863508,3000.000000,1385.501355
96070,6302977522903470578,2015-05-28T00:00:00+03:00,2017-04-26T00:00:00+03:00,550000.0,6,False,4,False,180.0,18.0,122.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3055.555556,1442.307692
96075,4377907628678297857,2016-11-07T00:00:00+03:00,2017-03-13T00:00:00+03:00,600000.0,6,False,4,False,187.0,NaN,127.0,100.0,1.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3208.556150,1442.307692
96091,1896730477252952293,2016-10-31T00:00:00+03:00,2017-05-26T00:00:00+03:00,650000.0,6,False,4,False,190.0,20.0,102.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3421.052632,1442.307692
154614,9174606537144832259,2017-11-16T00:00:00+03:00,2018-08-20T00:00:00+03:00,600000.0,1,False,4,False,165.0,NaN,115.0,NaN,6.0,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355,3636.363636,3636.363636
154615,8867625833179319559,2018-04-21T00:00:00+03:00,2018-08-09T00:00:00+03:00,600000.0,1,False,4,False,117.0,20.0,83.0,0.0,NaN,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355,5128.205128,3636.363636
154617,5152264856292210972,2018-02-01T00:00:00+03:00,2018-08-16T00:00:00+03:00,600000.0,1,False,4,False,140.0,95.0,100.0,100.0,NaN,2,2,"Россия, Санкт-Петербург, проспект Динамо, 6",135575281136523355,4285.714286,3636.363636


In [37]:
# look at offers with price > 1 000 000
rent_df_cleaned[(rent_df_cleaned.last_price >= 1000000)]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
67005,5199154,2016-01-25T00:00:00+03:00,2016-01-26T00:00:00+03:00,3800000.0,3,False,2,False,51.0,6.0,30.5,NaN,0.0,2,2,"Россия, Санкт-Петербург, Пискарёвский проспект, 149",7757194067653498771,74509.803922,74509.803922
95935,8743101202301731585,2016-10-13T00:00:00+03:00,2016-11-12T00:00:00+03:00,1500000.0,3,False,2,False,54.7,NaN,NaN,NaN,7.0,2,2,"Россия, Ленинградская область, Выборгский район, Светогорское городское поселение, поселок городского типа Лесогорский, улица Труда, 7",2755098623625145659,27422.303473,13811.700183
114024,4075481786711135488,2016-08-18T00:00:00+03:00,2016-08-18T00:00:00+03:00,3500000.0,1,False,2,False,55.0,8.3,30.5,NaN,3.0,2,2,"Россия, Ленинградская область, Всеволожск, микрорайон Южный, Центральная улица, 13",4521154714631889065,63636.363636,63636.363636
164662,4088017598264857345,2018-02-09T00:00:00+03:00,2018-03-21T00:00:00+03:00,1330000.0,3,False,1,False,33.0,6.0,19.0,NaN,3.0,2,2,"Россия, Ленинградская область, Луга, проспект Урицкого, 65",1792010144615909485,40303.030303,40303.030303


In [38]:
# we see, that all of them are outliers, they were not cleaned by the house price per sq m median 
# because it was the only one or just one of two offers per house
# let's clean them also
# Store result in the same dataframe
rent_df_cleaned = rent_df_cleaned[rent_df_cleaned.last_price < 1000000]

In [39]:
# let's examine apartments with big price per sq m
rent_df_cleaned[rent_df_cleaned.price_per_sq_m > 3000]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
549,8596035942924139265,2018-05-17T00:00:00+03:00,2018-07-04T00:00:00+03:00,450000.0,6,False,2,False,106.0,15.0,65.0,30.0,NaN,2,2,"Россия, Санкт-Петербург, Воскресенская набережная, 4",7262659557693263963,4245.283019,975.609756
28182,1492050389917501301,2017-06-07T00:00:00+03:00,2017-06-07T00:00:00+03:00,195000.0,4,False,1,False,57.0,8.0,19.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, проспект Добролюбова, 2",6684242979485775735,3421.052632,779.220779
39213,3793550987719225600,2017-03-19T00:00:00+03:00,2017-11-02T00:00:00+03:00,310000.0,3,False,4,False,100.0,14.0,71.0,100.0,1.0,2,2,"Россия, Санкт-Петербург, Лиговский проспект, 123",2861706125080798064,3100.000000,976.307190
44574,1896730477284436579,2017-05-31T00:00:00+03:00,2017-05-31T00:00:00+03:00,230000.0,9,False,2,False,67.0,14.0,30.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Глухая Зеленина улица, 6",6445545084936583797,3432.835821,923.076923
96070,6302977522903470578,2015-05-28T00:00:00+03:00,2017-04-26T00:00:00+03:00,550000.0,6,False,4,False,180.0,18.0,122.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3055.555556,1442.307692
96075,4377907628678297857,2016-11-07T00:00:00+03:00,2017-03-13T00:00:00+03:00,600000.0,6,False,4,False,187.0,NaN,127.0,100.0,1.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3208.556150,1442.307692
96091,1896730477252952293,2016-10-31T00:00:00+03:00,2017-05-26T00:00:00+03:00,650000.0,6,False,4,False,190.0,20.0,102.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3421.052632,1442.307692
99409,7102677476104578816,2016-05-20T00:00:00+03:00,2016-05-20T00:00:00+03:00,270000.0,1,False,1,False,80.0,NaN,NaN,0.0,3.0,2,2,"Россия, Санкт-Петербург, Загородный проспект, 5",8810747501928407960,3375.000000,1978.197674
104668,159653575951184384,2016-06-16T00:00:00+03:00,2016-07-01T00:00:00+03:00,300000.0,2,False,3,False,54.0,8.0,35.0,50.0,11.0,2,2,"Россия, Санкт-Петербург, 18-я линия В.О., 43",6003296125598219505,5555.555556,5555.555556
150701,1492050390917641037,2016-10-28T00:00:00+03:00,2017-06-07T00:00:00+03:00,400000.0,4,False,3,False,130.0,13.0,85.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Константиновский проспект, 23",5091810811209082905,3076.923077,3484.848485


In [40]:
# we can see that outliers are those which are in houses with price per sq m below 1000 rubles, 
# and those which are the same with median, in which case it's the only offer per house
outliers_count = len(rent_df_cleaned[(rent_df_cleaned.price_per_sq_m > 3000) 
                                     & ((rent_df_cleaned.house_price_sqm_median < 1000) 
                                        | (rent_df_cleaned.house_price_sqm_median == rent_df_cleaned.price_per_sq_m))])
print("outliers found: {}".format(outliers_count))
rent_df_cleaned = rent_df_cleaned[~((rent_df_cleaned.price_per_sq_m > 3000) 
                                     & ((rent_df_cleaned.house_price_sqm_median < 1000) 
                                        | (rent_df_cleaned.house_price_sqm_median == rent_df_cleaned.price_per_sq_m)))]

outliers found: 8


In [41]:
# now max price should be correct, lets look at this offer
max_rent_apartment = rent_df_cleaned[rent_df_cleaned.last_price == rent_df_cleaned.last_price.max()]
max_rent_apartment

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
96091,1896730477252952293,2016-10-31T00:00:00+03:00,2017-05-26T00:00:00+03:00,650000.0,6,False,4,False,190.0,20.0,102.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Парадная улица, 3к1",7861463534572863507,3421.052632,1442.307692


In [42]:
# let's look at house address. 
# with 'iloc' property we can take any row by index, 
# in this case we take the first item to get the address
max_rent_apartment.unified_address.iloc[0]

'Россия, Санкт-Петербург, Парадная улица, 3к1'

In [43]:
# impoort urllib to prepare correct URLs for looking at offers archive 
# at https://realty.yandex.ru/archive
import urllib

In [44]:
# if you're interested in images of this apartment you can find it here: 
# https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20Парадная%20улица%2C%203к1/snyat/kvartira/2/?roomsTotal=4

# let's print the URL of the page, on which we can find our offer
# we use urllib.parse.quote function to get URL part from our address, otherwise URL wouldn't be correct
# print("https://realty.yandex.ru/archive/{}/snyat/kvartira/2/?roomsTotal=4".format(urllib.parse.quote(max_rent_apartment.unified_address.iloc[0])))
# print("https://realty.yandex.ru/otsenka-kvartiry-po-adresu-onlayn/{}/snyat/kvartira/?roomsTotal=4".format(urllib.parse.quote(max_rent_apartment.unified_address.iloc[0])))
print("https://realty.yandex.ru/otsenka-kvartiry-po-adresu-onlayn/{}/snyat/kvartira/".format(urllib.parse.quote(max_rent_apartment.unified_address.iloc[0])))
# use filters there to select only apartments with 4 rooms

https://realty.yandex.ru/otsenka-kvartiry-po-adresu-onlayn/%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F%2C%20%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3%2C%20%D0%9F%D0%B0%D1%80%D0%B0%D0%B4%D0%BD%D0%B0%D1%8F%20%D1%83%D0%BB%D0%B8%D1%86%D0%B0%2C%203%D0%BA1/snyat/kvartira/


### Find houses with the most cheapest and the most expensive rent price

In [45]:
# Create a new dataframe with houses and median price per sq m
# This time it should with the data calculated without outliers
house_rent_df_cleaned = rent_df_cleaned.groupby('unified_address').price_per_sq_m.median().reset_index()
house_rent_df_cleaned.rename(columns = {'price_per_sq_m': 'house_price_sqm_median_cleaned'}, inplace = True)

In [46]:
most_expensive_house = house_rent_df_cleaned[house_rent_df_cleaned.house_price_sqm_median_cleaned == house_rent_df_cleaned.house_price_sqm_median_cleaned.max()]
most_expensive_house

,unified_address,house_price_sqm_median_cleaned
16851,"Россия, Санкт-Петербург, проспект Динамо, 6",3705.357143


In [47]:
# you can look at the most expensive house here
# https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20проспект%20Динамо%2C%206/snyat/kvartira/
print("https://realty.yandex.ru/archive/{}/snyat/kvartira/".format(urllib.parse.quote(most_expensive_house.unified_address.iloc[0])))

https://realty.yandex.ru/archive/%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F%2C%20%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3%2C%20%D0%BF%D1%80%D0%BE%D1%81%D0%BF%D0%B5%D0%BA%D1%82%20%D0%94%D0%B8%D0%BD%D0%B0%D0%BC%D0%BE%2C%206/snyat/kvartira/


#### Find the most cheapest house for rent

In [48]:
most_cheapest_house = house_rent_df_cleaned[house_rent_df_cleaned.house_price_sqm_median_cleaned == house_rent_df_cleaned.house_price_sqm_median_cleaned.min()]
most_cheapest_house

,unified_address,house_price_sqm_median_cleaned
2421,"Россия, Ленинградская область, Кириши, улица Мира, 14",50.0


In [49]:
# we see that it's located in Leningrad Oblast, let's look only in St. Petersburg data
house_rent_df_cleaned_spb = house_rent_df_cleaned[house_rent_df_cleaned.unified_address.str.contains('Россия, Санкт-Петербург')]
house_rent_df_cleaned_spb[house_rent_df_cleaned_spb.house_price_sqm_median_cleaned == house_rent_df_cleaned_spb.house_price_sqm_median_cleaned.min()]

,unified_address,house_price_sqm_median_cleaned
4852,"Россия, Санкт-Петербург, Большая Зеленина улица, 12",76.923077


#### Clean cheap outliers
We see that we should clean the data, the cheapest house above is definitely a mistake, and it is an underground station building. You can check it at https://maps.yandex.ru

In [50]:
# consider only data inside the city without oblast
rent_df_cleaned_spb = rent_df_cleaned[rent_df_cleaned.unified_address.str.contains('Россия, Санкт-Петербург')]
median_price_per_sq_m_in_spb = rent_df_cleaned_spb.price_per_sq_m.median()
print("Median price per sq m in St. Petersburg in rent: {}".format(median_price_per_sq_m_in_spb))

Median price per sq m in St. Petersburg in rent: 555.5555555555555


In [51]:
# look at cheap apartments which are more than two times cheaper 
# than the median value of offers we calculated above (555)
rent_df_cleaned_spb[rent_df_cleaned_spb.price_per_sq_m < 250].sample(10)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
101233,4468447061739523841,2017-12-19T00:00:00+03:00,2018-03-21T00:00:00+03:00,16500.0,8,False,3,False,75.0,NaN,NaN,80.0,NaN,2,2,"Россия, Санкт-Петербург, Красное Село, Кингисеппское шоссе, 8",7603381608587612443,220.000000,220.000000
120882,604446981317765377,2018-03-05T00:00:00+03:00,2018-04-04T00:00:00+03:00,17000.0,5,False,3,False,80.0,NaN,NaN,50.0,NaN,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Ростовская улица, 14-16",4153179922410625475,212.500000,264.912281
85324,2600779294684148225,2016-04-07T00:00:00+03:00,2016-04-24T00:00:00+03:00,15000.0,5,False,2,False,63.0,12.0,40.0,60.0,10.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, Колпинское шоссе, 61",7452048126409559398,238.095238,333.307692
131648,5172709105186092032,2018-04-21T00:00:00+03:00,2018-07-29T00:00:00+03:00,16000.0,7,False,2,False,65.0,NaN,NaN,50.0,NaN,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Ростовская улица, 26к1",9059817452517931591,246.153846,310.344828
165487,1680370125623308289,2018-03-23T00:00:00+03:00,2018-04-19T00:00:00+03:00,19000.0,2,False,3,False,81.0,NaN,NaN,50.0,NaN,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Полоцкая улица, 13к2",305237777299840432,234.567901,225.000000
73246,8703508311489187328,2016-07-24T00:00:00+03:00,2016-08-02T00:00:00+03:00,16000.0,6,False,3,False,80.0,8.0,43.0,50.0,3.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, Колпинское шоссе, 40",7452048126409559335,200.000000,377.777778
76836,956953260097037825,2018-05-26T00:00:00+03:00,2018-06-24T00:00:00+03:00,22000.0,2,False,1,False,99.0,NaN,NaN,NaN,NaN,2,2,"Россия, Санкт-Петербург, проспект Энгельса, 113к2",6711366030409491511,222.222222,590.909091
13824,7569479969705953273,2018-02-20T00:00:00+03:00,2018-02-20T00:00:00+03:00,20000.0,11,False,2,False,90.0,10.0,NaN,99.0,NaN,2,2,"Россия, Санкт-Петербург, Ленинский проспект, 79к1",1861668167106596765,222.222222,460.839280
120846,1570606376708378368,2017-11-29T00:00:00+03:00,2018-02-06T00:00:00+03:00,19000.0,8,False,3,False,82.0,14.0,48.0,0.0,11.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Ростовская улица, 14-16",4153179922410625475,231.707317,264.912281
123853,6361132933439615488,2017-11-03T00:00:00+03:00,2018-02-10T00:00:00+03:00,17000.0,5,False,3,False,80.0,11.0,44.0,NaN,3.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Ростовская улица, 3к2",897860344996780193,212.500000,316.666667


In [52]:
# it seems that we should clean all apartments where median price is at least 2x higher and cheap 
outliers = rent_df_cleaned_spb[(rent_df_cleaned_spb.price_per_sq_m < 250) 
                               & (rent_df_cleaned_spb.house_price_sqm_median/rent_df_cleaned_spb.price_per_sq_m >= 2)]

In [53]:
print("number of cheap outliers 2x cheaper: {}".format(len(outliers)))

number of cheap outliers 2x cheaper: 100


In [54]:
# clean outliers
rent_df_cleaned_spb = rent_df_cleaned_spb[~((rent_df_cleaned_spb.price_per_sq_m < 250) 
                               & (rent_df_cleaned_spb.house_price_sqm_median/rent_df_cleaned_spb.price_per_sq_m >= 2))]

In [55]:
# look at cheap apartments which were not cleaned with the previous heuristics
rent_df_cleaned_spb[rent_df_cleaned_spb.price_per_sq_m < 200]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
46867,4857633,2015-12-13T00:00:00+03:00,2016-06-05T00:00:00+03:00,12000.0,6,False,3,False,61.0,7.0,42.0,80.0,3.0,2,2,"Россия, Санкт-Петербург, Красное Село, Стрельнинское шоссе, 6к3",1523555400005784785,196.721311,256.578947
57861,4975278,2015-12-27T00:00:00+03:00,2016-01-12T00:00:00+03:00,13000.0,4,False,3,False,78.0,10.0,47.0,NaN,10.0,2,2,"Россия, Санкт-Петербург, 12-я Красноармейская улица, 7",6204918944542168630,166.666667,166.666667
80404,2939523434608526081,2018-03-06T00:00:00+03:00,2018-03-13T00:00:00+03:00,15000.0,6,False,3,False,87.0,NaN,NaN,NaN,3.0,2,2,"Россия, Санкт-Петербург, Окраинная улица, 9В",4510539508387660402,172.413793,241.715116
85844,249021112485953840,2016-03-21T00:00:00+03:00,2016-07-05T00:00:00+03:00,12000.0,3,False,4,False,112.0,14.0,16.0,NaN,0.0,2,2,"Россия, Санкт-Петербург, Свеаборгская улица, 23",4943588578198944211,107.142857,107.142857
91302,8176284694677069056,2016-05-18T00:00:00+03:00,2016-05-18T00:00:00+03:00,14000.0,4,False,3,False,74.0,10.0,NaN,50.0,3.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, квартал Славянка, Ростовская улица, 6к3",4976238169808254780,189.189189,291.963016
101249,5379909360861020417,2016-06-02T00:00:00+03:00,2016-06-09T00:00:00+03:00,14000.0,3,False,3,False,80.0,12.0,NaN,80.0,0.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, квартал Славянка, Ростовская улица, 27",5194982532050895016,175.000000,233.333333
105186,801981697508642560,2016-06-20T00:00:00+03:00,2016-06-29T00:00:00+03:00,12000.0,6,False,5,False,156.0,23.0,16.0,50.0,11.0,2,2,"Россия, Санкт-Петербург, Большая Зеленина улица, 12",7137635118733633172,76.923077,76.923077
105957,6302976505020612958,2016-06-25T00:00:00+03:00,2016-11-08T00:00:00+03:00,17000.0,4,False,3,False,100.0,16.0,20.0,50.0,0.0,2,2,"Россия, Санкт-Петербург, Центральный район, улица Некрасова, 38",4677861887331789522,170.000000,170.000000
106615,3653332947260070401,2016-07-01T00:00:00+03:00,2016-07-08T00:00:00+03:00,11000.0,3,False,2,False,71.0,9.0,42.0,50.0,3.0,2,2,"Россия, Санкт-Петербург, Красное Село, Нагорная улица, 37",4011485438398100794,154.929577,183.098592
120839,2164881834443110913,2017-05-07T00:00:00+03:00,2017-05-14T00:00:00+03:00,15000.0,5,False,3,False,93.0,10.0,54.0,100.0,0.0,2,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Славянка, Ростовская улица, 14-16",4153179922410625475,161.290323,264.912281


In [56]:
# there are cases where houses had just one offer and house median equals offer's price per square meter
# let's remove these outliers
rent_df_cleaned_spb = rent_df_cleaned_spb[~((rent_df_cleaned_spb.price_per_sq_m < 200) 
                                          & (rent_df_cleaned_spb.price_per_sq_m == rent_df_cleaned_spb.house_price_sqm_median))]

In [57]:
# look at the cheapest apartment
cheapest_offer = rent_df_cleaned_spb[rent_df_cleaned_spb.price_per_sq_m == rent_df_cleaned_spb.price_per_sq_m.min()]
cheapest_offer

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
106615,3653332947260070401,2016-07-01T00:00:00+03:00,2016-07-08T00:00:00+03:00,11000.0,3,False,2,False,71.0,9.0,42.0,50.0,3.0,2,2,"Россия, Санкт-Петербург, Красное Село, Нагорная улица, 37",4011485438398100794,154.929577,183.098592


#### Recalculate houses price medians on the cleaned data and find the cheapest house

In [58]:
house_rent_df_cleaned_spb = rent_df_cleaned_spb.groupby('unified_address').price_per_sq_m.median().reset_index()
house_rent_df_cleaned_spb.rename(columns = {'price_per_sq_m': 'house_price_sqm_median_cleaned'}, inplace = True)
cheapest_house = house_rent_df_cleaned_spb[house_rent_df_cleaned_spb.house_price_sqm_median_cleaned == house_rent_df_cleaned_spb.house_price_sqm_median_cleaned.min()]
cheapest_house

,unified_address,house_price_sqm_median_cleaned
5245,"Россия, Санкт-Петербург, Красное Село, Нагорная улица, 37",183.098592


In [59]:
# looking at the map we understand why it's so cheap: it's far away from the center of the city
# https://realty.yandex.ru/archive/Россия%2C%20Санкт-Петербург%2C%20Красное%20Село%2C%20Нагорная%20улица%2C%2037/snyat/kvartira/
print("https://realty.yandex.ru/archive/{}/snyat/kvartira/".format(urllib.parse.quote(cheapest_house.unified_address.iloc[0])))

https://realty.yandex.ru/archive/%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F%2C%20%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3%2C%20%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%BE%D0%B5%20%D0%A1%D0%B5%D0%BB%D0%BE%2C%20%D0%9D%D0%B0%D0%B3%D0%BE%D1%80%D0%BD%D0%B0%D1%8F%20%D1%83%D0%BB%D0%B8%D1%86%D0%B0%2C%2037/snyat/kvartira/


### Analyze how many offers have the commission 

In [60]:
# Some offers has commision fee for the agent (agent_fee)
# With 'value_counts' function find out the share of corresponding value
rent_df_cleaned_spb.agent_fee.value_counts(normalize=True, dropna=False)

100.000000    0.328129
50.000000     0.293380
NaN           0.209340
0.000000      0.035378
90.000000     0.034537
                ...   
14.000000     0.000006
11.000000     0.000006
17.000000     0.000006
92.000000     0.000006
64.997665     0.000006
Name: agent_fee, Length: 102, dtype: float64

We see that the most popular commission is 100% - 33% of offers. 21% of offers don't have any information about commission.

### Self-control stops
1. Calculate median and mean prices for apartments for rent after cleaning the data in St.Petersburg without Leningrad Oblast. Which of the statistics changed more and why?
2. Calculate median and mean prices for apartments for sell before cleaning the data.
2. Find houses with the most cheapest and most expensive price per sq m in St. Petersburg without Leningrad Oblast after cleaning outliers.
3. Find the most expensive and the most cheapest apartment in St. Petersburg after cleaning outliers.
4. Calculate how many years does it take to cover all money spent on buying apartment by renting it. Find houses in which it's the most effective to invest in buying if you plan to rent and houses in which it will be the most ineffective.

**1. Calculate median and mean prices for apartments for rent after cleaning the data in St.Petersburg without Leningrad Oblast. Which of the statistics changed more and why?**

In [61]:
rent_df_cleaned_spb

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
0,1100202,2015-01-24T00:00:00+03:00,2016-01-19T00:00:00+03:00,20000.0,7,False,1,False,28.0,8.0,20.0,100.0,3.0,2,2,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",6719200042030752141,714.285714,512.820513
1,4593125,2015-11-17T00:00:00+03:00,2016-03-04T00:00:00+03:00,24000.0,4,False,2,False,59.0,10.0,35.0,100.0,3.0,2,2,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",6719200042030752141,406.779661,512.820513
2,4586184,2015-11-17T00:00:00+03:00,2016-04-24T00:00:00+03:00,18000.0,6,False,1,False,36.0,9.0,17.0,100.0,3.0,2,2,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",6719200042030752141,500.000000,512.820513
3,5308071,2016-02-04T00:00:00+03:00,2016-02-28T00:00:00+03:00,18000.0,8,False,1,False,39.0,10.0,18.0,90.0,0.0,2,2,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",6719200042030752141,461.538462,512.820513
4,7534283612538021121,2016-02-28T00:00:00+03:00,2016-04-02T00:00:00+03:00,19000.0,3,False,1,False,36.0,8.0,18.0,50.0,11.0,2,2,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",6719200042030752141,527.777778,512.820513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171180,5152264856294776681,2018-08-20T00:00:00+03:00,2018-08-20T00:00:00+03:00,26000.0,2,False,1,False,32.0,8.0,24.0,100.0,NaN,2,2,"Россия, Санкт-Петербург, Приморский проспект, 54к4",1218123990061225954,812.500000,812.500000
171182,5141797489725174647,2018-08-20T00:00:00+03:00,2018-08-20T00:00:00+03:00,27000.0,1,False,1,False,39.0,8.0,20.0,100.0,NaN,2,2,"Россия, Санкт-Петербург, улица Подковырова, 31",4039469598438558938,692.307692,692.307692
171183,8000484694230616001,2018-08-20T00:00:00+03:00,2018-08-20T00:00:00+03:00,30000.0,3,False,2,False,67.0,NaN,NaN,50.0,NaN,2,2,"Россия, Санкт-Петербург, улица Савушкина, 46",2213552294614164759,447.761194,447.761194
171184,8335109787193152729,2018-08-20T00:00:00+03:00,2018-08-20T00:00:00+03:00,50000.0,2,False,2,False,56.0,7.0,32.0,50.0,NaN,2,2,"Россия, Санкт-Петербург, Казанская улица, 56",1703651731789727211,892.857143,892.857143


In [62]:
rent_median_price = rent_df_cleaned_spb.last_price.median()
print("Rent median price: {}".format(rent_median_price))

rent_mean_price = rent_df_cleaned_spb.last_price.mean()
print("Rent mean price: {}".format(rent_mean_price))

Rent median price: 25000.0
Rent mean price: 32605.087189685033


We see the difference in mean because of deleated outliers

**2. Calculate median and mean prices for apartments for sell before cleaning the data**

In [63]:
sell_median_price = sell_df.last_price.median()
print("Sell median price: {}".format(sell_median_price))

sell_mean_price = sell_df.last_price.mean()
print("Sell mean price: {}".format(sell_mean_price))

Sell median price: 4599000.0
Sell mean price: 6969288.146034318


**3. Find houses with the most cheapest and most expensive price per sq m in St. Petersburg without Leningrad Oblast after cleaning outliers**

In [64]:
max_price_sell = sell_df.last_price.max()
print("Maximum price: {}".format(max_price_sell))

min_price_sell = sell_df.last_price.min()
print("Minimum price: {}".format(min_price_sell))

mean_price_sell = sell_df.last_price.mean()
print("Mean price: {}".format(mean_price_sell))

median_price_sell = sell_df.last_price.median()
print("Median price: {}".format(median_price_sell))

Maximum price: 763000000.0
Minimum price: 19000.0
Mean price: 6969288.146034318
Median price: 4599000.0


In [65]:
sell_df[sell_df.last_price == max_price_sell]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
224434,619165944711730944,2017-09-30T00:00:00+03:00,2017-11-02T00:00:00+03:00,763000000.0,10,False,7,False,400.0,NaN,250.0,NaN,1.0,1,2,"Россия, Санкт-Петербург, Мичуринская улица, 4",7962972499404580673


In [66]:
sell_df[sell_df.last_price == min_price_sell]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id
70001,3438368921946229505,2016-06-16T00:00:00+03:00,2016-06-19T00:00:00+03:00,19000.0,4,False,1,True,39.0,NaN,18.0,NaN,11.0,1,2,"Россия, Санкт-Петербург, улица Ленсовета, 69к1",8047169421659571687


In [67]:
# Create a new column price_per_sq_m and calculate price per sq m
sell_df['price_per_sq_m'] = sell_df.last_price/sell_df.area

/tmp/ipykernel_456/3164263733.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sell_df['price_per_sq_m'] = sell_df.last_price/sell_df.area


In [68]:
sell_df.head(5)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m
0,5490504238877583424,2014-11-27T00:00:00+03:00,2016-03-14T00:00:00+03:00,3676740.0,19,False,1,False,27.40,16.56,16.56,NaN,0.0,1,2,"Россия, Санкт-Петербург, улица Шкапина, 9-11",3552413344225333068,134187.591241
1,5490504238903444021,2014-11-27T00:00:00+03:00,2016-04-28T00:00:00+03:00,7038880.0,2,False,3,False,85.84,19.64,41.20,NaN,0.0,1,2,"Россия, Санкт-Петербург, Кушелевская дорога, 3к4",7274736308217971906,82000.000000
2,5490504238903447743,2014-11-27T00:00:00+03:00,2016-05-11T00:00:00+03:00,7038880.0,3,False,3,False,85.84,19.64,41.20,NaN,0.0,1,2,"Россия, Санкт-Петербург, Кушелевская дорога, 3к4",7274736308217971906,82000.000000
3,5490504238877673946,2014-11-27T00:00:00+03:00,2016-05-12T00:00:00+03:00,12074400.0,5,False,3,False,93.60,10.90,52.00,NaN,0.0,1,2,"Россия, Санкт-Петербург, Свердловская набережная, 58А",5917543958366039686,129000.000000
4,5490504238877648740,2014-11-27T00:00:00+03:00,2016-05-12T00:00:00+03:00,20406100.0,7,False,4,False,142.70,13.84,82.55,NaN,0.0,1,2,"Россия, Санкт-Петербург, Свердловская набережная, 58А",5917543958366039686,143000.000000


In [69]:
median_price_per_sq_m = sell_df.price_per_sq_m.median()
mean_price_per_sq_m = sell_df.price_per_sq_m.mean()
print("Median price per sq m in sell: {}".format(median_price_per_sq_m))
print("Mean price per sq m in sell: {}".format(mean_price_per_sq_m))

Median price per sq m in sell: 95214.28571428571
Mean price per sq m in sell: 102713.88007995683


In [70]:
house_sell_df = sell_df.groupby('unified_address').price_per_sq_m.median().reset_index()

In [71]:
house_sell_df.head(5)

,unified_address,price_per_sq_m
0,"Россия, Ленинградская область, Бокситогорск, Городская улица, 1",15316.249610
1,"Россия, Ленинградская область, Бокситогорск, Городская улица, 3",19080.327869
2,"Россия, Ленинградская область, Бокситогорск, Городская улица, 4",20812.534664
3,"Россия, Ленинградская область, Бокситогорск, Дымское шоссе, 1",14600.000000
4,"Россия, Ленинградская область, Бокситогорск, Заводская улица, 11",14642.978290


In [72]:
house_sell_df.rename(columns = {'price_per_sq_m': 'house_price_sqm_median'}, inplace = True)

In [73]:
sell_df = sell_df.merge(house_sell_df)

In [74]:
outliers = sell_df[(sell_df.price_per_sq_m/sell_df.house_price_sqm_median) > 5]

In [75]:
print(len(outliers))

52


In [76]:
outliers.sample(10)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
91005,3331592112315918037,2015-11-23T00:00:00+03:00,2016-01-07T00:00:00+03:00,118000000.0,11,False,3,False,120.00,17.00,63.00,NaN,0.0,1,2,"Россия, Санкт-Петербург, проспект Луначарского, 11к1",3271810337494402643,9.833333e+05,118421.052632
170482,3367195771329321292,2016-06-09T00:00:00+03:00,2016-09-13T00:00:00+03:00,86000000.0,3,False,4,False,75.20,8.30,53.60,NaN,0.0,1,2,"Россия, Санкт-Петербург, Думская улица, 5/22",7300988142191185101,1.143617e+06,148706.896552
120479,5152132572093402629,2016-12-23T00:00:00+03:00,2016-12-26T00:00:00+03:00,92894184.0,3,False,1,False,45.59,11.86,20.47,NaN,0.0,1,2,"Россия, Ленинградская область, Всеволожский район, Колтушское сельское поселение, деревня Старая, Школьный переулок, 3",3653851415933114236,2.037600e+06,55100.000000
122264,1693814676007144193,2017-03-23T00:00:00+03:00,2017-09-04T00:00:00+03:00,43800000.0,4,False,2,False,46.00,7.00,28.00,NaN,7.0,1,2,"Россия, Санкт-Петербург, проспект Луначарского, 110",3675879831604894982,9.521739e+05,87617.021277
78920,2403652735985551105,2017-11-26T00:00:00+03:00,2017-12-05T00:00:00+03:00,11490000.0,2,True,0,False,14.40,NaN,9.20,NaN,2.0,1,2,"Россия, Санкт-Петербург, улица Ленина, 8",580801932174144010,7.979167e+05,140113.636364
15996,2547228509556334337,2016-12-17T00:00:00+03:00,2017-03-18T00:00:00+03:00,30900000.0,6,False,1,False,36.10,10.30,15.20,NaN,3.0,1,2,"Россия, Ленинградская область, Всеволожский район, деревня Новое Девяткино, Арсенальная улица, 1",1966219418622870602,8.559557e+05,80555.555556
247432,1278395478499908865,2018-08-16T00:00:00+03:00,2018-08-20T00:00:00+03:00,25000000.0,5,False,1,False,25.00,NaN,NaN,NaN,NaN,1,2,"Россия, Ленинградская область, Всеволожский район, Заневское городское поселение, Кудрово, Столичная улица, 4к3",3877292615414429742,1.000000e+06,111542.224898
106330,7195375218410563072,2017-11-28T00:00:00+03:00,2018-08-02T00:00:00+03:00,118000000.0,2,False,3,False,98.30,12.50,51.00,NaN,NaN,1,2,"Россия, Санкт-Петербург, Киевская улица, 6",601237935667676085,1.200407e+06,158298.246523
124501,216145565898896384,2016-12-20T00:00:00+03:00,2017-01-19T00:00:00+03:00,59401500.0,5,False,3,False,55.00,6.00,39.00,NaN,3.0,1,2,"Россия, Санкт-Петербург, Ленинский проспект, 172",5890508792580294584,1.080027e+06,89263.392857
133336,8457217352584277359,2016-05-24T00:00:00+03:00,2016-05-29T00:00:00+03:00,37500000.0,2,False,2,False,43.00,NaN,NaN,NaN,0.0,1,2,"Россия, Санкт-Петербург, Суздальский проспект, 77к1",7092781136104321151,8.720930e+05,87187.500000


In [77]:
sell_df_cleaned = sell_df[~((sell_df.price_per_sq_m/sell_df.house_price_sqm_median) > 5)]

In [78]:
sell_df_cleaned[sell_df_cleaned.price_per_sq_m > 700000]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
5183,5028522700597151473,2015-06-18T00:00:00+03:00,2017-01-09T00:00:00+03:00,199208000.0,6,False,4,False,230.9,132.4,68.7,NaN,0.0,1,2,"Россия, Санкт-Петербург, Конногвардейский бульвар, 5",8043782038424496855,8.627458e+05,4.009150e+05
5202,5867283855213039065,2015-09-03T00:00:00+03:00,2016-12-05T00:00:00+03:00,199208976.0,6,False,4,False,230.9,132.4,72.3,NaN,0.0,1,2,"Россия, Санкт-Петербург, Конногвардейский бульвар, 5",8043782038424496855,8.627500e+05,4.009150e+05
5240,244178420220903129,2015-03-26T00:00:00+03:00,2016-11-15T00:00:00+03:00,199208976.0,6,False,5,False,231.0,20.0,184.7,NaN,0.0,1,2,"Россия, Санкт-Петербург, Конногвардейский бульвар, 5",8043782038424496855,8.623765e+05,4.009150e+05
5359,1357736667257575168,2016-03-30T00:00:00+03:00,2016-09-18T00:00:00+03:00,200000000.0,8,True,0,False,250.0,NaN,185.0,NaN,0.0,1,2,"Россия, Санкт-Петербург, Таврическая улица, 35",8255041317048885714,8.000000e+05,2.084485e+05
7427,5028522700594591427,2015-01-30T00:00:00+03:00,2017-01-09T00:00:00+03:00,132840000.0,3,False,3,False,147.6,20.0,95.6,NaN,0.0,1,2,"Россия, Санкт-Петербург, Большая Конюшенная улица, 10",2673376664769372453,9.000000e+05,3.699546e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240274,8310251267999680038,2018-02-26T00:00:00+03:00,2018-08-18T00:00:00+03:00,240000000.0,3,False,2,False,195.9,16.2,48.2,2.0,NaN,1,2,"Россия, Санкт-Петербург, Мытнинская набережная, 5",6697739012327356614,1.225115e+06,3.395586e+05
252215,8310251191879356399,2017-07-10T00:00:00+03:00,2018-08-20T00:00:00+03:00,109900000.0,2,False,4,False,140.2,44.3,79.7,0.0,1.0,1,2,"Россия, Санкт-Петербург, Депутатская улица, 8",7168777433308944355,7.838802e+05,7.838802e+05
252557,4454015731391335937,2017-10-16T00:00:00+03:00,2018-08-20T00:00:00+03:00,95000000.0,6,False,3,False,117.5,10.0,84.0,NaN,10.0,1,2,"Россия, Санкт-Петербург, Каменноостровский проспект, 35А",8273788561911339386,8.085106e+05,4.865373e+05
252644,3094343811119509760,2017-11-20T00:00:00+03:00,2018-08-20T00:00:00+03:00,95000000.0,6,False,3,False,117.5,10.0,84.0,NaN,NaN,1,2,"Россия, Санкт-Петербург, Каменноостровский проспект, 35/75",564112227981355965,8.085106e+05,8.085106e+05


In [79]:
outliers_count = len(sell_df_cleaned[(sell_df_cleaned.price_per_sq_m > 500000) 
                                     & ((sell_df_cleaned.house_price_sqm_median < 200000) 
                                        | (sell_df_cleaned.house_price_sqm_median == sell_df_cleaned.price_per_sq_m))])
print("outliers found: {}".format(outliers_count))
sell_df_cleaned = sell_df_cleaned[~((sell_df_cleaned.price_per_sq_m > 500000) 
                                     & ((sell_df_cleaned.house_price_sqm_median < 200000) 
                                        | (sell_df_cleaned.house_price_sqm_median == sell_df_cleaned.price_per_sq_m)))]

outliers found: 49


In [80]:
max_sell_apartment = sell_df_cleaned[sell_df_cleaned.last_price == sell_df_cleaned.last_price.max()]
max_sell_apartment

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
237351,8310251211233707984,2018-03-26T00:00:00+03:00,2018-08-20T00:00:00+03:00,605740032.0,2,False,5,False,443.0,10.0,248.0,0.0,NaN,1,2,"Россия, Санкт-Петербург, Южная дорога, 5",3312559599398519904,1.367359e+06,759301.442673


In [81]:
max_sell_apartment.unified_address.iloc[0]

'Россия, Санкт-Петербург, Южная дорога, 5'

In [82]:
house_sell_df_cleaned = sell_df_cleaned.groupby('unified_address').price_per_sq_m.median().reset_index()
house_sell_df_cleaned.rename(columns = {'price_per_sq_m': 'house_price_sqm_median_cleaned'}, inplace = True)

In [83]:
most_expensive_house = house_sell_df_cleaned[house_sell_df_cleaned.house_price_sqm_median_cleaned == house_sell_df_cleaned.house_price_sqm_median_cleaned.max()]
most_expensive_house

,unified_address,house_price_sqm_median_cleaned
23002,"Россия, Санкт-Петербург, Санаторная аллея, 3",818152.92298


In [84]:
most_cheapest_house = house_sell_df_cleaned[house_sell_df_cleaned.house_price_sqm_median_cleaned == house_sell_df_cleaned.house_price_sqm_median_cleaned.min()]
most_cheapest_house

,unified_address,house_price_sqm_median_cleaned
16228,"Россия, Санкт-Петербург, Конторская улица, 16",5000.0


In [85]:
house_sell_df_cleaned_spb = house_sell_df_cleaned[house_sell_df_cleaned.unified_address.str.contains('Россия, Санкт-Петербург')]
house_sell_df_cleaned_spb[house_sell_df_cleaned_spb.house_price_sqm_median_cleaned == house_sell_df_cleaned_spb.house_price_sqm_median_cleaned.min()]

,unified_address,house_price_sqm_median_cleaned
16228,"Россия, Санкт-Петербург, Конторская улица, 16",5000.0


In [86]:
sell_df_cleaned_spb = sell_df_cleaned[sell_df_cleaned.unified_address.str.contains('Россия, Санкт-Петербург')]
median_price_per_sq_m_in_spb = sell_df_cleaned_spb.price_per_sq_m.median()
print("Median price per sq m in St. Petersburg in sell: {}".format(median_price_per_sq_m_in_spb))

Median price per sq m in St. Petersburg in sell: 102272.72727272728


In [87]:
sell_df_cleaned_spb[sell_df_cleaned_spb.price_per_sq_m < (median_price_per_sq_m_in_spb/2)].sample(10)

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
40720,8980173618296520193,2016-04-18T00:00:00+03:00,2016-06-02T00:00:00+03:00,1580000.0,2,False,5,False,110.0,9.2,70.00,NaN,0.0,1,2,"Россия, Санкт-Петербург, проспект Стачек, 26",4203185621775190147,14363.636364,86516.321279
137473,1345900549831789057,2016-07-21T00:00:00+03:00,2018-07-16T00:00:00+03:00,3300000.0,4,False,2,False,70.0,15.0,41.00,NaN,10.0,1,2,"Россия, Санкт-Петербург, Колпинский район, посёлок Понтонный, Заводская улица, 18",6103388847815428933,47142.857143,55537.728027
54637,4625448,2015-11-20T00:00:00+03:00,2016-01-04T00:00:00+03:00,1480000.0,5,False,1,False,100.0,17.0,76.00,NaN,3.0,1,2,"Россия, Санкт-Петербург, Московский проспект, 79",4901520549922120767,14800.000000,108333.333333
188500,6043818824623828480,2016-05-29T00:00:00+03:00,2016-06-07T00:00:00+03:00,1400000.0,2,False,5,False,173.0,19.0,14.50,NaN,0.0,1,2,"Россия, Санкт-Петербург, Можайская улица, 10",6479857151474488258,8092.485549,8092.485549
228951,7110103192797396791,2017-05-19T00:00:00+03:00,2017-05-23T00:00:00+03:00,2100000.0,2,False,3,False,68.0,7.0,24.00,NaN,0.0,1,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, территория Пулковское, 32",7136212736664716937,30882.352941,68269.230769
169230,3076272492544922881,2017-06-09T00:00:00+03:00,2017-07-24T00:00:00+03:00,3350000.0,15,False,2,False,70.0,14.0,35.00,NaN,3.0,1,2,"Россия, Санкт-Петербург, посёлок Парголово, улица Первого Мая, 107к1",8685649315768240630,47857.142857,59000.000000
243900,5542542669723237632,2017-11-29T00:00:00+03:00,2018-01-13T00:00:00+03:00,1890000.0,3,False,1,False,44.0,12.4,23.32,NaN,NaN,1,2,"Россия, Санкт-Петербург, Рябовское шоссе, 130",6712797249654250853,42954.545455,42954.545455
185551,7388659598552164096,2018-02-01T00:00:00+03:00,2018-05-12T00:00:00+03:00,2870000.0,7,False,2,False,60.0,10.0,32.00,NaN,NaN,1,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, Первомайская улица, 5к3",6931516357826986190,47833.333333,62500.000000
78947,5709288895787653377,2016-08-24T00:00:00+03:00,2016-10-08T00:00:00+03:00,1250000.0,4,False,0,True,197.1,NaN,118.70,NaN,0.0,1,2,"Россия, Санкт-Петербург, улица Стойкости, 36к1",132296357019531359,6341.958397,7918.781726
184181,5122598656777858560,2018-04-20T00:00:00+03:00,2018-08-20T00:00:00+03:00,2900000.0,7,False,2,False,60.0,13.0,30.00,NaN,3.0,1,2,"Россия, Санкт-Петербург, посёлок Парголово, Тихоокеанская улица, 1к1",2421317494945802192,48333.333333,66274.076002


In [88]:
outliers = sell_df_cleaned_spb[(sell_df_cleaned_spb.price_per_sq_m < (median_price_per_sq_m_in_spb/2)) 
                               & (sell_df_cleaned_spb.house_price_sqm_median/sell_df_cleaned_spb.price_per_sq_m >= 2)]

In [89]:
print("number of cheap outliers 2x cheaper: {}".format(len(outliers)))

number of cheap outliers 2x cheaper: 305


In [90]:
sell_df_cleaned_spb = sell_df_cleaned_spb[~((sell_df_cleaned_spb.price_per_sq_m < (median_price_per_sq_m_in_spb/2)) 
                               & (sell_df_cleaned_spb.house_price_sqm_median/sell_df_cleaned_spb.price_per_sq_m >= 2))]

In [91]:
sell_df_cleaned_spb[sell_df_cleaned_spb.price_per_sq_m < (median_price_per_sq_m_in_spb/2)]

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
10990,9069199666252787288,2014-12-10T00:00:00+03:00,2016-09-03T00:00:00+03:00,6390000.0,3,False,3,False,125.0,14.30,67.30,NaN,0.0,1,2,"Россия, Санкт-Петербург, Ломоносов, Еленинская улица, 24",7877007895598460542,51120.000000,53581.871345
10994,9069199666280455402,2016-08-05T00:00:00+03:00,2016-09-28T00:00:00+03:00,4050000.0,4,False,2,False,100.0,14.00,64.00,NaN,0.0,1,2,"Россия, Санкт-Петербург, Ломоносов, Еленинская улица, 24",7877007895598460542,40500.000000,53581.871345
11683,8353202309096040645,2017-06-02T00:00:00+03:00,2018-02-07T00:00:00+03:00,3600000.0,3,False,3,False,76.0,6.20,54.20,NaN,0.0,1,2,"Россия, Санкт-Петербург, Колпинский район, посёлок Понтонный, улица Судостроителей, 9",5318902771108014783,47368.421053,56907.894737
11919,9174606537143044917,2015-06-24T00:00:00+03:00,2017-02-07T00:00:00+03:00,2200000.0,4,False,2,False,53.0,6.00,29.00,NaN,0.0,1,2,"Россия, Санкт-Петербург, Курортный район, посёлок Молодёжное, улица Правды, 15",2185806163382678206,41509.433962,71186.440678
13652,4255749407203426048,2016-09-26T00:00:00+03:00,2016-10-10T00:00:00+03:00,3804000.0,5,False,2,False,81.0,11.31,46.16,NaN,0.0,1,2,"Россия, Санкт-Петербург, Красносельское шоссе, 56к3",8961386408260776630,46962.962963,75829.383886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257312,384297957418301953,2018-07-23T00:00:00+03:00,2018-08-11T00:00:00+03:00,2100000.0,1,False,1,False,42.0,10.00,18.00,NaN,3.0,1,2,"Россия, Санкт-Петербург, набережная канала Грибоедова, 94",3976567739161800104,50000.000000,50000.000000
257482,4763425724156481025,2018-08-02T00:00:00+03:00,2018-08-20T00:00:00+03:00,1750000.0,1,False,1,False,43.6,NaN,NaN,NaN,NaN,1,2,"Россия, Санкт-Петербург, Сестрорецк, улица Володарского, 31",1373968259503776925,40137.614679,40137.614679
257598,8056737725713679872,2018-08-12T00:00:00+03:00,2018-08-20T00:00:00+03:00,2600000.0,1,False,2,False,52.8,6.70,32.70,NaN,2.0,1,2,"Россия, Санкт-Петербург, посёлок Лисий Нос, Ивановская улица, 12к4",2183408768595820797,49242.424242,49242.424242
257927,5177752565079589149,2018-02-01T00:00:00+03:00,2018-08-20T00:00:00+03:00,2050000.0,1,False,3,False,55.0,7.20,37.40,NaN,NaN,1,2,"Россия, Санкт-Петербург, посёлок Левашово, улица Кирова, 2",620571518529338168,37272.727273,37272.727273


In [92]:
sell_df_cleaned_spb = sell_df_cleaned_spb[~((sell_df_cleaned_spb.price_per_sq_m < (median_price_per_sq_m_in_spb/2)) 
                                          & (sell_df_cleaned_spb.price_per_sq_m == sell_df_cleaned_spb.house_price_sqm_median))]

In [93]:
cheapest_offer = sell_df_cleaned_spb[sell_df_cleaned_spb.price_per_sq_m == sell_df_cleaned_spb.price_per_sq_m.min()]
cheapest_offer

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
78947,5709288895787653377,2016-08-24T00:00:00+03:00,2016-10-08T00:00:00+03:00,1250000.0,4,False,0,True,197.1,NaN,118.7,NaN,0.0,1,2,"Россия, Санкт-Петербург, улица Стойкости, 36к1",132296357019531359,6341.958397,7918.781726


In [94]:
expensive_offer = sell_df_cleaned_spb[sell_df_cleaned_spb.price_per_sq_m == sell_df_cleaned_spb.price_per_sq_m.max()]
expensive_offer

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
7438,8310251211319744238,2018-04-04T00:00:00+03:00,2018-08-20T00:00:00+03:00,330000000.0,5,False,3,False,190.0,40.0,95.0,0.0,1.0,1,2,"Россия, Санкт-Петербург, Большая Конюшенная улица, 10",2673376664769372453,1.736842e+06,369954.557605
7439,8310251211319744363,2018-04-04T00:00:00+03:00,2018-08-20T00:00:00+03:00,330000000.0,6,False,3,False,190.0,25.0,90.0,0.0,1.0,1,2,"Россия, Санкт-Петербург, Большая Конюшенная улица, 10",2673376664769372453,1.736842e+06,369954.557605


**4. Find the most expensive and the most cheapest apartment in St. Petersburg after cleaning outliers**

In [95]:
cheapest_apart = sell_df_cleaned_spb[sell_df_cleaned_spb.last_price == sell_df_cleaned_spb.last_price.min()]
cheapest_apart

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
185561,5365283220161389569,2018-04-24T00:00:00+03:00,2018-04-27T00:00:00+03:00,1100000.0,1,False,0,True,33.0,NaN,NaN,NaN,3.0,1,2,"Россия, Санкт-Петербург, Пушкинский район, посёлок Шушары, Первомайская улица, 5к3",6931516357826986190,33333.333333,62500.0


In [96]:
expensive_apart = sell_df_cleaned_spb[sell_df_cleaned_spb.last_price == sell_df_cleaned_spb.last_price.max()]
expensive_apart

,offer_id,first_day_exposition,last_day_exposition,last_price,floor,open_plan,rooms,studio,area,kitchen_area,living_area,agent_fee,renovation,offer_type,category_type,unified_address,building_id,price_per_sq_m,house_price_sqm_median
237351,8310251211233707984,2018-03-26T00:00:00+03:00,2018-08-20T00:00:00+03:00,605740032.0,2,False,5,False,443.0,10.0,248.0,0.0,NaN,1,2,"Россия, Санкт-Петербург, Южная дорога, 5",3312559599398519904,1.367359e+06,759301.442673


**5. Calculate how many years does it take to cover all money spent on buying apartment by renting it. Find houses in which it's the most effective to invest in buying if you plan to rent and houses in which it will be the most ineffective**

In [97]:
sel=sell_df_cleaned_spb[['building_id','unified_address','last_price']]
sel

,building_id,unified_address,last_price
0,3552413344225333068,"Россия, Санкт-Петербург, улица Шкапина, 9-11",3676740.0
1,3552413344225333068,"Россия, Санкт-Петербург, улица Шкапина, 9-11",3285640.0
2,3552413344225333068,"Россия, Санкт-Петербург, улица Шкапина, 9-11",4669280.0
3,3552413344225333068,"Россия, Санкт-Петербург, улица Шкапина, 9-11",4539560.0
4,3552413344225333068,"Россия, Санкт-Петербург, улица Шкапина, 9-11",4509750.0
...,...,...,...
257994,3434079020108720685,"Россия, Санкт-Петербург, улица Шелгунова, 24",3650000.0
257996,442073173549249451,"Россия, Санкт-Петербург, улица Ткачей, 64",5550000.0
257997,5720923704279729581,"Россия, Санкт-Петербург, переулок Ульяны Громовой, 8Б",5600000.0
257998,8515757547030833919,"Россия, Санкт-Петербург, проспект Энгельса, 137",6200000.0


In [98]:
sel_gr = sel.groupby(['building_id','unified_address','last_price']).mean().astype(int).reset_index()
sel_gr.columns = ['building_id','unified_address','av_price_sel']
sel_gr

,building_id,unified_address,av_price_sel
0,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",4200000.0
1,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",4800000.0
2,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",4990000.0
3,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",5200000.0
4,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",6000000.0
...,...,...,...
174435,9220297187880756244,"Россия, Санкт-Петербург, Якорная улица, 1к2",3950000.0
174436,9220297187880756244,"Россия, Санкт-Петербург, Якорная улица, 1к2",4100000.0
174437,9220297187880756244,"Россия, Санкт-Петербург, Якорная улица, 1к2",4160000.0
174438,9220297187880756244,"Россия, Санкт-Петербург, Якорная улица, 1к2",4340000.0


In [99]:
ren=rent_df_cleaned_spb[['building_id','unified_address','last_price']]
ren['price_per_year']=ren['last_price']*12
ren

/tmp/ipykernel_456/2033641083.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ren['price_per_year']=ren['last_price']*12


,building_id,unified_address,last_price,price_per_year
0,6719200042030752141,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",20000.0,240000.0
1,6719200042030752141,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",24000.0,288000.0
2,6719200042030752141,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",18000.0,216000.0
3,6719200042030752141,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",18000.0,216000.0
4,6719200042030752141,"Россия, Санкт-Петербург, Богатырский проспект, 5к1",19000.0,228000.0
...,...,...,...,...
171180,1218123990061225954,"Россия, Санкт-Петербург, Приморский проспект, 54к4",26000.0,312000.0
171182,4039469598438558938,"Россия, Санкт-Петербург, улица Подковырова, 31",27000.0,324000.0
171183,2213552294614164759,"Россия, Санкт-Петербург, улица Савушкина, 46",30000.0,360000.0
171184,1703651731789727211,"Россия, Санкт-Петербург, Казанская улица, 56",50000.0,600000.0


In [100]:
ren_gr = ren.groupby(['building_id','unified_address'])['price_per_year'].mean().astype(int).reset_index()
ren_gr.columns = ['building_id','unified_address','av_price_ren']
ren_gr

,building_id,unified_address,av_price_ren
0,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",330000
1,1785805724177444,"Россия, Санкт-Петербург, Рыбацкая улица, 12А",540000
2,3286884802341379,"Россия, Санкт-Петербург, Пушкинский район, Пушкин, Гренадерская улица, 12к2",216000
3,3603962364551559,"Россия, Санкт-Петербург, Учебный переулок, 8к3",322000
4,3603962364551560,"Россия, Санкт-Петербург, Учебный переулок, 8к2",360000
...,...,...,...
19841,9217640295586864077,"Россия, Санкт-Петербург, Гороховая улица, 17/56",684000
19842,9217640295587698130,"Россия, Санкт-Петербург, Гороховая улица, 24/24",660000
19843,9217640295588649554,"Россия, Санкт-Петербург, Гороховая улица, 35-37",410000
19844,9218305025595159775,"Россия, Санкт-Петербург, Соляной переулок, 16",696000


In [101]:
m_df = pd.merge(ren_gr,sel_gr,on=['building_id','unified_address'], how='inner')
m_df['cover']=(m_df['av_price_sel']/m_df['av_price_ren']).round(2)
m_df

,building_id,unified_address,av_price_ren,av_price_sel,cover
0,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",330000,4200000.0,12.73
1,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",330000,4800000.0,14.55
2,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",330000,4990000.0,15.12
3,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",330000,5200000.0,15.76
4,458573008430269,"Россия, Санкт-Петербург, Биржевая линия, 1",330000,6000000.0,18.18
...,...,...,...,...,...
149119,9220297187880756244,"Россия, Санкт-Петербург, Якорная улица, 1к2",282600,3950000.0,13.98
149120,9220297187880756244,"Россия, Санкт-Петербург, Якорная улица, 1к2",282600,4100000.0,14.51
149121,9220297187880756244,"Россия, Санкт-Петербург, Якорная улица, 1к2",282600,4160000.0,14.72
149122,9220297187880756244,"Россия, Санкт-Петербург, Якорная улица, 1к2",282600,4340000.0,15.36


In [102]:
m_df.sort_values(by='cover', ascending=True).head(5)

,building_id,unified_address,av_price_ren,av_price_sel,cover
62946,3967985793187418353,"Россия, Санкт-Петербург, Боровая улица, 94",1800000,2230000.0,1.24
43526,2654887542667114864,"Россия, Санкт-Петербург, улица Чехова, 11-13",1200000,1755000.0,1.46
87190,5553660887271054733,"Россия, Санкт-Петербург, Средний проспект Васильевского острова, 27",1680000,2500000.0,1.49
72425,4570380531975985556,"Россия, Санкт-Петербург, 11-я линия Васильевского острова, 18",960000,1430000.0,1.49
72626,4635282170202574216,"Россия, Санкт-Петербург, проспект Чернышевского, 9",1800000,2730000.0,1.52
